<a href="https://colab.research.google.com/github/GL54/ML-train-animals/blob/main/learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Lambda, Dense, Flatten

from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from glob import glob
import numpy as np

import pandas as pd
import seaborn as sns


from matplotlib import pyplot as plt

In [ ]:
IMAGE_SIZE=[224,224]

PREPROCESSING

In [ ]:
train_directory= '/content/dataset/raw-img'
test_directory = '/content/dataset/training_set'

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights = 'imagenet', include_top= False )

for layer in vgg.layers:
  layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
folders = glob('/content/dataset/raw-img/*')
len(folders)

10

In [ ]:
x=Flatten()(vgg.output)
prediction = Dense(len(folders),activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

In [ ]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True
                                   )

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(train_directory,
                                                 target_size=(224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical'
                                                 )
test_set = test_datagen.flow_from_directory(test_directory,
                                            target_size=(224,224),
                                            batch_size = 32,
                                            class_mode='categorical'
                                            )

Found 26179 images belonging to 10 classes.
Found 148 images belonging to 10 classes.


In [ ]:
print(len(training_set))

819


In [ ]:
r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=1,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)

)

<ipython-input-19-349af39fbdb1>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


819/819 [==============================] - 16090s 20s/step - loss: 0.8128 - accuracy: 0.7498 - val_loss: 0.5917 - val_accuracy: 0.8108


In [ ]:
model.save('BC.h5')

In [ ]:
from keras.models import load_model
from keras_preprocessing.image import load_img , img_to_array

model1  = load_model('./BC.h5',compile = False)

In [ ]:
lab = training_set.class_indices
lab = {k:v for v,k in lab.items()}
lab


NameError: ignored

In [ ]:
def output(location):
  img = load_img(location,target_size=(224,224,3))
  img = img_to_array(img)
  img = img/225
  img = np.expand_dims(img,[0])
  answer = model1.predict(img)
  y_class = answer.argmax(axis =-1)
  y = " ".join(str(x) for x in y_class)
  y = int(y)
  res = lab[y]
  return res

In [ ]:
img = '/content/a.jpg'

output(img)

1/1 [==============================] - 1s 1s/step


'ragno'

In [ ]:
! pip install keras_preprocessing



In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d alessiocorrado99/animals10

In [ ]:
!unzip animals10.zip -d /content/dataset

In [ ]:
!unzip training_set.zip -d /content/dataset

In [ ]:
data = pd.read_csv('/content/csv.csv')
data 

,translations,label_names
0,cane,dog
1,cavallo,horse
2,elefante,elephant
3,farfalla,butterfly
4,gallina,chicken
5,gatto,cat
6,mucca,cow
7,pecora,sheep
8,scoiattolo,squirrel
9,ragno,spider
